In [ ]:
import sys
import os
sys.path.append("/content/ml-project-2-pytyeee/")

REPO_DIR = "ml-project-2-pytyeee/"

In [1]:
from scripts.preprocessing import *
from helpers.model_handler import *

ModuleNotFoundError: No module named 'scripts'

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
test_dir = REPO_DIR + "dataset/test_set_images/"
test_images_filename = test_dir

TESTING_SIZE = 50

test_data = extract_data(test_images_filename, TESTING_SIZE, test=True)
test_data = torch.from_numpy(np.transpose(test_data, (0, 3, 1, 2)))
print(f"Test shape: {test_data.shape}")

test_data = test_data.to(DEVICE)

In [ ]:
predictions = model(test_data)
pred = torch.where(predictions > 0.5, 1., 0.)